In [1]:
%matplotlib notebook
%matplotlib inline
import os

OK_PATH='/Data_Storage/Rui_Data_Space/VQA/OK-VQA'
import pandas as pd
from collections import defaultdict

In [2]:
import json
import pickle as pkl

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [3]:
from PIL import Image
import os
import random

In [4]:
val_anno=load_json(os.path.join(OK_PATH,'mscoco_val2014_annotations.json'))['annotations']
ans_dict={}
for row in val_anno:
    ques_id=str(row['question_id'])
    answers=defaultdict(int)
    for info in row['answers']:
        answers[info['answer']]+=1
    #ans={ans:answers[ans]/sum(answers.values()) for ans in answers.keys()}
    ans_dict[ques_id]=answers
print (len(val_anno),len(ans_dict))

5046 5046


In [5]:
vqa_all=load_json(os.path.join(OK_PATH,'OpenEnded_mscoco_val2014_questions.json'))['questions']
print (len(vqa_all))

5046


In [6]:
def get_score(occurences):
    return min(1.0,occurences/3.0)

import inflect
single = inflect.engine()

def norm_ans(ans):
    if ans.endswith(',') or ans.endswith('.'):
        ans=ans[:-1]
    if ans.startswith('a '):
        ans=ans[2:]
    if ',' in ans:
        ans=ans.split(',')[0]
    if len(ans.split(' '))==1 and ans not in ['grass','glass','bus']:
        norm_ans=single.singular_noun(ans)
        if norm_ans !=False:
            ans=norm_ans
    return ans

In [7]:
import torch
CUDA_DEVICE=14
torch.cuda.set_device(CUDA_DEVICE)
device = torch.device("cuda:"+str(CUDA_DEVICE))

In [8]:
from lavis.common.gradcam import getAttMap
from lavis.models import load_model_and_preprocess
model, vis_processors, txt_processors = load_model_and_preprocess(name="pnp_vqa",
                                                                  model_type="3b", 
                                                                  is_eval=True,device=device)

In [10]:
NUM_CAP=65
vis=0
NUM_KB=0
total={}
acc=0.0

for k,row in enumerate(vqa_all):
    if k%200==0:
        print ('Already finished:',
               k,k*100.0/len(vqa_all) )
        if k>0:
            print ('\n\tAcc:',acc*100.0/vis)
    ques_id=str(row['question_id'])
        
    img_id='COCO_val2014_'+str(row['image_id']).zfill(12)+'.jpg'
    im=Image.open(os.path.join(OK_PATH,'images',img_id)).convert('RGB')
    image = vis_processors["eval"](im).unsqueeze(0).to(device)
    text=row['question']
    question = txt_processors["eval"](text)
    samples = {"image": image, "text_input": [question]}
    captions=load_json('../OK_VQA/large_captions/'+ques_id+'.json')
    samples['captions']=[captions[:NUM_CAP]]
    
    if NUM_KB>0:
        kb=load_json(os.path.join('../OK_VQA/cluster_generated_kb',
                                  ques_id+'.json'))
        kb=kb[:NUM_KB]
        if len(' '.join(kb).split(' '))>500:
            kb=kb[:20]
        samples['captions'][0].extend(kb)
        
    ans=ans_dict[ques_id]
        
    pred_answers = model.forward_qa(samples,
                                    num_captions=NUM_CAP+NUM_KB)
    pred=norm_ans(pred_answers[0].lower())
    total[ques_id]=pred
    """print (text,ans_dict[ques_id])
    print (pred)"""
    vis+=1
    if pred in ans_dict[ques_id]:
        acc+=get_score(ans_dict[ques_id][pred])
print (acc*100.0/vis)
print (vis)
pkl.dump(total,
         open('pnp_ablations_'+'_'.join([str(NUM_CAP),
                                         str(NUM_KB)])+'.pkl','wb'))

Already finished: 0 0.0
Already finished: 200 3.963535473642489

	Acc: 37.16666666666666
Already finished: 400 7.927070947284978

	Acc: 34.08333333333333
Already finished: 600 11.890606420927467

	Acc: 32.3333333333333
Already finished: 800 15.854141894569956

	Acc: 32.91666666666662
Already finished: 1000 19.817677368212447

	Acc: 32.466666666666676
Already finished: 1200 23.781212841854934

	Acc: 32.38888888888893
Already finished: 1400 27.744748315497425

	Acc: 33.119047619047684
Already finished: 1600 31.708283789139912

	Acc: 33.79166666666673
Already finished: 1800 35.6718192627824

	Acc: 33.629629629629655
Already finished: 2000 39.635354736424894

	Acc: 33.91666666666665
Already finished: 2200 43.59889021006738

	Acc: 34.318181818181785
Already finished: 2400 47.56242568370987

	Acc: 34.51388888888883
Already finished: 2600 51.525961157352356

	Acc: 34.897435897435805
Already finished: 2800 55.48949663099485

	Acc: 34.9642857142856
Already finished: 3000 59.45303210463734

	Acc

In [10]:
NUM_CAP=65
vis=0
NUM_KB=5
total={}
acc=0.0

for k,row in enumerate(vqa_all):
    if k%200==0:
        print ('Already finished:',
               k,k*100.0/len(vqa_all) )
        if k>0:
            print ('\n\tAcc:',acc*100.0/vis)
    ques_id=str(row['question_id'])
        
    img_id='COCO_val2014_'+str(row['image_id']).zfill(12)+'.jpg'
    im=Image.open(os.path.join(OK_PATH,'images',img_id)).convert('RGB')
    image = vis_processors["eval"](im).unsqueeze(0).to(device)
    text=row['question']
    question = txt_processors["eval"](text)
    samples = {"image": image, "text_input": [question]}
    captions=load_json('../OK_VQA/large_captions/'+ques_id+'.json')
    samples['captions']=[captions[:NUM_CAP]]
    
    if NUM_KB>0:
        kb=load_json(os.path.join('../OK_VQA/cluster_generated_kb',
                                  ques_id+'.json'))
        kb=kb[:NUM_KB]
        samples['captions'][0].extend(kb)
        
    ans=ans_dict[ques_id]
        
    pred_answers = model.forward_qa(samples,
                                    num_captions=NUM_CAP+NUM_KB)
    pred=norm_ans(pred_answers[0].lower())
    total[ques_id]=pred
    """print (text,ans_dict[ques_id])
    print (pred)"""
    vis+=1
    if pred in ans_dict[ques_id]:
        acc+=get_score(ans_dict[ques_id][pred])
print (acc*100.0/vis)
print (vis)
pkl.dump(total,
         open('pnp_ablations_'+'_'.join([str(NUM_CAP),
                                         str(NUM_KB)])+'.pkl','wb'))

Already finished: 0 0.0
Already finished: 200 3.963535473642489

	Acc: 39.0
Already finished: 400 7.927070947284978

	Acc: 36.00000000000001
Already finished: 600 11.890606420927467

	Acc: 35.33333333333332
Already finished: 800 15.854141894569956

	Acc: 36.12499999999999
Already finished: 1000 19.817677368212447

	Acc: 36.63333333333338
Already finished: 1200 23.781212841854934

	Acc: 36.83333333333341
Already finished: 1400 27.744748315497425

	Acc: 37.309523809523895
Already finished: 1600 31.708283789139912

	Acc: 37.89583333333335
Already finished: 1800 35.6718192627824

	Acc: 37.999999999999986
Already finished: 2000 39.635354736424894

	Acc: 38.24999999999996
Already finished: 2200 43.59889021006738

	Acc: 38.136363636363576
Already finished: 2400 47.56242568370987

	Acc: 38.13888888888879
Already finished: 2600 51.525961157352356

	Acc: 38.30769230769219
Already finished: 2800 55.48949663099485

	Acc: 38.59523809523802
Already finished: 3000 59.45303210463734

	Acc: 38.81111111

In [12]:
NUM_CAP=65
vis=0
NUM_KB=10
total={}
acc=0.0

for k,row in enumerate(vqa_all):
    if k%200==0:
        print ('Already finished:',
               k,k*100.0/len(vqa_all) )
        if k>0:
            print ('\n\tAcc:',acc*100.0/vis)
    ques_id=str(row['question_id'])
        
    img_id='COCO_val2014_'+str(row['image_id']).zfill(12)+'.jpg'
    im=Image.open(os.path.join(OK_PATH,'images',img_id)).convert('RGB')
    image = vis_processors["eval"](im).unsqueeze(0).to(device)
    text=row['question']
    question = txt_processors["eval"](text)
    samples = {"image": image, "text_input": [question]}
    captions=load_json('../OK_VQA/large_captions/'+ques_id+'.json')
    samples['captions']=[captions[:NUM_CAP]]
    
    if NUM_KB>0:
        kb=load_json(os.path.join('../OK_VQA/cluster_generated_kb',
                                  ques_id+'.json'))
        kb=kb[:NUM_KB]
        samples['captions'][0].extend(kb)
        
    ans=ans_dict[ques_id]
        
    pred_answers = model.forward_qa(samples,
                                    num_captions=NUM_CAP+NUM_KB)
    pred=norm_ans(pred_answers[0].lower())
    total[ques_id]=pred
    """print (text,ans_dict[ques_id])
    print (pred)"""
    vis+=1
    if pred in ans_dict[ques_id]:
        acc+=get_score(ans_dict[ques_id][pred])
print (acc*100.0/vis)
print (vis)
pkl.dump(total,
         open('pnp_ablations_'+'_'.join([str(NUM_CAP),
                                         str(NUM_KB)])+'.pkl','wb'))

Already finished: 0 0.0
Already finished: 200 3.963535473642489

	Acc: 40.500000000000014
Already finished: 400 7.927070947284978

	Acc: 38.25000000000002
Already finished: 600 11.890606420927467

	Acc: 36.61111111111111
Already finished: 800 15.854141894569956

	Acc: 37.91666666666668
Already finished: 1000 19.817677368212447

	Acc: 38.16666666666672
Already finished: 1200 23.781212841854934

	Acc: 38.41666666666675
Already finished: 1400 27.744748315497425

	Acc: 38.76190476190483
Already finished: 1600 31.708283789139912

	Acc: 39.083333333333336
Already finished: 1800 35.6718192627824

	Acc: 39.148148148148124
Already finished: 2000 39.635354736424894

	Acc: 39.31666666666661
Already finished: 2200 43.59889021006738

	Acc: 39.21212121212115
Already finished: 2400 47.56242568370987

	Acc: 39.04166666666657
Already finished: 2600 51.525961157352356

	Acc: 39.243589743589624
Already finished: 2800 55.48949663099485

	Acc: 39.39285714285708
Already finished: 3000 59.45303210463734

	Ac